# 3. vježba: modeliranje nizova povratnim neuronskim mrežama

In [1]:
from __future__ import print_function, division

import os
import sys
import re
import pdb
import time

import numpy as np
import scipy as sp
import tensorflow as tf

## Zadatak 1: učitavanje podataka i batching

In [61]:
# hardcoded paths - change if necessary
root = 'data'

# this one you need to download from the dataset
full_dataset = 'data/movie_lines.txt'

output_destination = 'data/selected_conversations.txt'
movie_selection = 'data/selected_movies.txt'

# separator used in the original dataset
separator = ' +++$+++ '

# movie ID file
MOVIE_ID = 0

# full conversation dataset file
MOVIE_ID_FULL = 2
# reverse indexing
CHARACTER_NAME = -2
CHARACTER_LINE = -1

# keep just these characters for simplicity (and utf8 breaking)
repl = r'[^A-Za-z0-9()\,!\?\'\`\. ]'


# regex replace
def filter(string):
    return re.sub(repl, '', string)


# from a movie ID string (e.g. M134), output the number (134)
def number_from_id(id):
    return int(id[1:])


# read just movie ID's, rest is for readability
def read_selected(path_to_selected_movies):
    selected_movies = set()

    with open(path_to_selected_movies, 'r') as infile:
        for line in infile:
            parts = line.strip().split(separator)
            selected_movies.add(parts[MOVIE_ID].strip())
    return selected_movies


# select and write to output file
def select_and_write(path_to_full_dataset, path_to_output, selected_movies):
    movies = {}

    with open(path_to_full_dataset, 'r', encoding="ISO-8859-1") as infile:

        for line in infile:

            parts = line.strip().split(separator)

            if parts[MOVIE_ID_FULL].strip() not in selected_movies:
                continue

            # take data and transform to tuple
            ID = parts[MOVIE_ID_FULL]
            char_name = parts[CHARACTER_NAME]
            char_line = parts[CHARACTER_LINE]

            tup = (number_from_id(ID), char_name, char_line)

            # add to map
            if ID not in movies:
                movies[ID] = []
            movies[ID].append(tup)

    with open(path_to_output, 'w') as out:
        for movie in movies:
            # sort by line number
            dialogue = sorted(movies[movie], key=lambda t: t[0])
            for n, name, text in dialogue:
                out.write(filter(name) + ':\n' + filter(text) + '\n\n')


def main():
    # uses hardcoded paths
    selection = os.path.join(root, movie_selection)
    selected_movies = read_selected(selection)

    dataset = os.path.join(root, full_dataset)
    output = os.path.join(root, output_destination)
    select_and_write(dataset, output, selected_movies)

In [62]:
class Dataset:
    def preprocess(self, input_file):
        with open(input_file, "r") as f:
            data = f.read()

        # count and sort most frequent characters
        characters, counts = np.unique(list(data), return_index=True)
        # self.sorted chars contains just the characters ordered descending by frequency
        self.sorted_characters = characters[np.argsort(-counts)]
        self.vocab_size = len(self.sorted_characters)

        self.char2id = dict(zip(self.sorted_characters, range(len(self.sorted_characters)))) 
        # reverse the mapping
        self.id2char = {k:v for v,k in self.char2id.items()}
        # convert the data to ids
        self.x = np.array(list(map(self.char2id.get, data)))

    def encode(self, sequence):
        encoded_sequence = np.array([self.char2id[c] for c in sequence])
        return encoded_sequence

    def decode(self, sequence):
        decoded_sequence = [self.id2char[c] for c in sequence]
        return decoded_sequence
        
    # creates minibatches with batch_size * sequence length letters
    def create_minibatches(self, batch_size, sequence_length):
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.batch_index = 0
        
        self.num_batches = int(len(self.x) / (batch_size * sequence_length)) # calculate the number of batches
        self.batches = np.zeros([self.num_batches, self.batch_size, self.sequence_length + 1], dtype=np.int32)  
    
        for minibatch_index in range(self.num_batches):
            for sequence_index in range(self.batch_size):
                start = minibatch_index * sequence_length + sequence_index * (self.num_batches * sequence_length)
                end = start + self.sequence_length + 1 
                self.batches[minibatch_index, sequence_index, :] = self.x[start : end]

    def next_minibatch(self):        
        if self.batch_index == self.num_batches:
            self.batch_index = 0

        batch = self.batches[self.batch_index, :, :]
        self.batch_index += 1
        
        batch_x = batch[:, :-1]
        batch_y = batch[:, 1:]
        
        return self.batch_index == self.num_batches, batch_x, batch_y
    
    def _as_one_hot(self, x, vocab):
        n = len(x)
        Yoh = np.zeros((n, vocab))
        Yoh[np.arange(n), x] = 1
        
        return Yoh
    
    def one_hot(self, batch):
        if batch.ndim == 1:
            return self._as_one_hot(batch, self.vocab_size)
        else:
            return np.array([self._as_one_hot(b, self.vocab_size) for b in batch])

In [63]:
# test 1
dat = Dataset()
dat.preprocess("data/selected_conversations.txt")

encoded = dat.encode("Day")
decoded = dat.decode(encoded)
print(dat.one_hot(encoded))
print(decoded)
print(dat.decode(dat.encode("Tomorrow")))



[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['D', 'a', 'y']
['T', 'o', 'm', 'o', 'r', 'r', 'o', 'w']


## Zadatak 2: obična jednoslojna povratna neuronska mreža

In [64]:
class RNN():
    # inicijalizacija parametara
    def __init__(self, hidden_size, sequence_length, vocabulary_size, learning_rate):
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.vocabulary_size = vocabulary_size
        self.learning_rate = learning_rate
        
        print("Number of parameters in the RNN: " + str(2*vocabulary_size*hidden_size + hidden_size + hidden_size*hidden_size + vocabulary_size))
        print("Hidden size: " + str(hidden_size))
        print("Sequence length: " + str(sequence_length))
        print("Vocabulary size: " + str(vocabulary_size))
        print("\n")
        
        self.U = np.random.normal(size=[vocabulary_size, hidden_size], scale=1.0 / np.sqrt(hidden_size))  # ... input projection
        self.W = np.random.normal(size=[hidden_size, hidden_size], scale=1.0 / np.sqrt(hidden_size))  # ... hidden-to-hidden projection
        self.b = np.zeros([1, hidden_size])

        self.V = np.random.normal(size=[hidden_size, vocabulary_size], scale=1.0 / np.sqrt(vocabulary_size)) # ... output projection
        self.c = np.zeros([1, vocabulary_size]) # ... output bias

        # memory of past gradients - rolling sum of squares for Adagrad
        self.memory_U = np.zeros_like(self.U)
        self.memory_W = np.zeros_like(self.W)
        self.memory_V = np.zeros_like(self.V)
        self.memory_b = np.zeros_like(self.b)
        self.memory_c = np.zeros_like(self.c)
        
    def rnn_step_forward(self, x, h_prev, U, W, b):
        # A single time step forward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        # x - input data (minibatch size x input dimension)
        # h_prev - previous hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)

        h_current, cache = None, None
        h_current = np.tanh(np.dot(h_prev, W) + np.dot(x, U) + b)
        cache = (W, x, h_prev, h_current)

        # return the new hidden state and a tuple of values needed for the backward step

        return h_current, cache


    def rnn_forward(self, x, h0, U, W, b):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        # x - input data for the whole time-series (minibatch size x sequence_length x input dimension)
        # h0 - initial hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)
        
        h, cache = None, None
        hs = [h0]
        caches = []
        
        for sequence in range(self.sequence_length):
            data = x[:, sequence, :]
            h_current, cache_current = self.rnn_step_forward(data, hs[-1], U, W, b)
            
            hs.append(h_current)
            caches.append(cache_current)
            
        # transposes the hs matrix according to indexes
        hs = np.array(hs[1:]).transpose((1, 0, 2))
        
        # return the hidden states for the whole time series (T+1) and a tuple of values needed for the backward step
        return hs, caches
    
    def rnn_step_backward(self, grad_next, cache):
        # A single time step backward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        # grad_next - upstream gradient of the loss with respect to the next hidden state and current output
        # cache - cached information from the forward pass        
        dh_prev, dU, dW, db = None, None, None, None
        W, x, h_prev, h_current = cache
        
        # compute and return gradients with respect to each parameter
        # HINT: you can use the chain rule to compute the derivative of the
        # hyperbolic tangent function and use it to compute the gradient
        # with respect to the remaining parameters
        
        # 65. i 66. slajd formule
        
        # gradijent tangensa hiperbolnog
        d_tanh = 1 - h_current**2
        
        da = grad_next * (d_tanh)
        
        # gradijenti na parametre
        dU = np.dot(x.T, da)
        dW = np.dot(h_prev.T, da)
        db = np.sum(da, axis=0)
        
        # gradijenti na ulaze
        dh_prev = np.dot(da, W.T)

        return dh_prev, dU, dW, db


    def rnn_backward(self, dh, cache):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        dU = np.zeros_like(self.U)
        dW = np.zeros_like(self.W)
        db = np.zeros_like(self.b)

        # compute and return gradients with respect to each parameter
        # for the whole time series.
        # Why are we not computing the gradient with respect to inputs (x)?
        # A: because we are using only one layer, there is nothing to propagate through the network
        grads = np.zeros_like(dh[0])
        for dh_t, cache_t in reversed(list(zip(dh, cache))):
            grads, dU_t, dW_t, db_t = self.rnn_step_backward(dh_t + grads, cache_t)
            
            dU += dU_t
            dW += dW_t
            db += db_t
        
        return np.clip(dU, -5, 5), np.clip(dW, -5, 5), np.clip(db, -5, 5)
    
    def softmax(self, x):
        x -= np.max(x)
        logits_exp = np.exp(x)
        return logits_exp / np.sum(logits_exp, axis=1, keepdims=True)
    
    def output(self, h, V, c):
        # Calculate the output 
        return np.dot(h, V) + c

    def output_loss_and_grads(self, h, V, c, y):
        # Calculate the loss of the network for each of the outputs

        # h - hidden states of the network for each timestep. 
        #     the dimensionality of h is (batch size x sequence length x hidden size (the initial state is irrelevant for the output)
        # V - the output projection matrix of dimension hidden size x vocabulary size
        # c - the output bias of dimension vocabulary size x 1
        # y - the true class distribution - a tensor of dimension 
        #     batch_size x sequence_length x vocabulary size - you need to do this conversion prior to
        #     passing the argument. A fast way to create a one-hot vector from
        #     an id could be something like the following code:

        #   y[batch_id][timestep] = np.zeros((vocabulary_size, 1))
        #   y[batch_id][timestep][batch_y[timestep]] = 1

        #     where y might be a list or a dictionary.

        loss, dh, dV, dc = None, None, None, None
        
        loss = 0
        dhs = []
        dV = np.zeros_like(self.V)
        dc = np.zeros_like(self.c)
        
        batch_size = len(h)
        
        # calculate the output (o) - unnormalized log probabilities of classes
        # calculate yhat - softmax of the output
        # calculate the cross-entropy loss
        # calculate the derivative of the cross-entropy softmax loss with respect to the output (o)
        # calculate the gradients with respect to the output parameters V and c
        # calculate the gradients with respect to the hidden layer h
        
        for sequence in range(self.sequence_length):
            y_true = y[:, sequence, :]
            h_t = h[:, sequence, :]
            
            o = self.output(h_t, V, c)
            yhat = self.softmax(o)
            
            # cross entropy log loss
            loss -= np.sum(np.log(yhat) * y_true) / batch_size
            
            dO = (yhat - y_true)
            dV += np.dot(h_t.T, dO)
            dc += np.sum(dO, axis=0)
            
            dhs.append(np.dot(dO, V.T))

        return loss, dhs, dV, dc
    
    # Adagrad parameter update
    def update_parameters(self, batch_size, dU, dW, db, dV, dc):
        # update memory matrices
        # perform the Adagrad update of parameters
        eps = 1e-7
        for x, dx, mem_x in zip([self.U, self.W, self.b, self.V, self.c], [dU, dW, db, dV, dc], [self.memory_U, self.memory_W, self.memory_b, self.memory_V, self.memory_c]):
            mem_x += np.square(dx)
            x -= self.learning_rate * dx / np.sqrt(mem_x + eps)
            
    def step(self, hs, x, y):
        hs, caches = self.rnn_forward(x, hs, self.U, self.W, self.b)
        loss, dh, dV, dc = self.output_loss_and_grads(hs, self.V, self.c, y)
        dU, dW, db = self.rnn_backward(dh, caches)
        
        self.update_parameters(len(x), dU, dW, db, dV, dc)
        
        # return loss and the last state value
        return loss, hs[:, -1, :]

In [71]:
def run_language_model(dataset, max_epochs, hidden_size=100, sequence_length=30, learning_rate=1e-1, sample_every=100):
    vocabulary_size = len(dataset.sorted_characters)
    rnn = RNN(hidden_size=hidden_size, sequence_length=sequence_length, vocabulary_size=vocabulary_size, learning_rate=learning_rate) # initialize the recurrent network

    current_epoch = 0 
    batch = 0

    h0 = np.zeros((dataset.batch_size, hidden_size))
    average_loss = 0
    
    should_print = False
    first_print = True
    while current_epoch < max_epochs: 
        e, x, y = dataset.next_minibatch()
        
        if e: 
            current_epoch += 1
            h0 = np.zeros((dataset.batch_size, hidden_size))
            
            current_batch = batch % dataset.num_batches
            print("Epoch: %06d" % (current_epoch), end="\n")
            print("Average_loss: %.4f; Last batch loss: %.4f" % (average_loss / batch, loss))
            print("=====================================================")
            
            should_print = True
            
        if first_print:
            seed = "HAN:\nIs that good or bad?\n\n"
            sampled_data = sample(rnn, seed, 300, dataset)

            print(''.join(sampled_data))
            print() 
            first_print = False
            

        # One-hot transform the x and y batches
        x_oh = dataset.one_hot(x)
        y_oh = dataset.one_hot(y)

        # Run the recurrent network on the current batch
        # Since we are using windows of a short length of characters,
        # the step function should return the hidden state at the end
        # of the unroll. You should then use that hidden state as the
        # input for the next minibatch. In this way, we artificially
        # preserve context between batches.
        loss, _ = rnn.step(h0, x_oh, y_oh)
        average_loss += loss

        if should_print: 
            # run sampling (2.2)
            seed = "HAN:\nIs that good or bad?\n\n"
            sampled_data = sample(rnn, seed, 300, dataset)

            print(''.join(sampled_data))
            print() 
            should_print = False
            
        batch += 1
        
def sample(rnn, seed, sample_size, dataset):
    h0 = np.zeros([1, rnn.hidden_size])
    seed_oh = dataset.one_hot(dataset.encode(seed))
    
    sampled = []
    
    for seed_character in seed_oh:
        h0, _ = rnn.rnn_step_forward(seed_character.reshape([1, -1]), h0, rnn.U, rnn.W, rnn.b)
        sampled.append(np.argmax(seed_character))
        
    for i in range(len(seed), sample_size):
        prev_out = np.array([sampled[-1]])
        in_oh = dataset.one_hot(prev_out)
        h0, _ = rnn.rnn_step_forward(in_oh, h0, rnn.U, rnn.W, rnn.b)
        
        probabilities = rnn.softmax(rnn.output(h0, rnn.V, rnn.c))
        # use random choice based on the probabilities, otherwise there will be repeating phrases
        out_char = np.random.choice(range(dataset.vocab_size), p=probabilities.ravel()) 
        sampled.append(out_char)
  
    return dataset.decode(sampled)

In [72]:
dataset = Dataset()
dataset.preprocess("data/selected_conversations.txt")
dataset.create_minibatches(batch_size=5, sequence_length=30)

run_language_model(dataset, 50, sequence_length=dataset.sequence_length)

Number of parameters in the RNN: 24371
Hidden size: 100
Sequence length: 30
Vocabulary size: 71


HAN:
Is that good or bad?

y!?xDq9i,wtpkO,:cltKt7O3aPuBB9MBNix3V5RkWnfEFd5KcpwDVipost6otHXwe?p,P5Yw6wyFXj'G7`XhEawPF.Au5pO6sYJenHrPzNlAJ6583uH
slixlIIHU?:YHO`Wp.4'ovndTig'rk fELFfNREraX'jQK?Lk::dbIRmYCXe1o0f

TtC
VCvNlI3FaNhEgz!leB:liPdDoRK8a v0y,x
1lv0nDnXvpfOApbOV1U.om6L`BUgjPJa6UfpbW!ccJSOZ,qkwKa

Epoch: 000001
Average_loss: 68.9526; Last batch loss: 57.8715
HAN:
Is that good or bad?

MECKARMqI'd courlived.

VEUGOR:
Mlen't buth, save.  Peely ertuve haves?

T Re wo f as hipim toued joll, gomsing bo the bere fulg? I filed bat is uBse, low. .n.

MANSOR:
Gise son then Tid pucl wor leot andaod asinotr that Coull huve in.

UKE:
I dut there lack bimid inoubeer

Epoch: 000002
Average_loss: 64.8254; Last batch loss: 55.0992
HAN:
Is that good or bad?

GHNDUTE:
Loan Car orvingcell won't about. Whyest a bare! I'le thind a got lanjy houve le.  tting, on ou do hin't...

LADED:
Where that tiser?

YANP

Epoch: 000020
Average_loss: 56.4588; Last batch loss: 48.3902
HAN:
Is that good or bad?

DONNBE:
Wo havisedn, you lets. Delet ar! Fist in?

DWIGHT:
That me.

DETOTHITE:
Jast about but ereterzandate suredtan whey? I's not and of the roge on.  Patbanf huuge.

GOTT:
Govn't sturty het angh.  Cong.?

DR. PBL:
And yNun't coun so goand you'll mach. What's all you's V

Epoch: 000021
Average_loss: 56.3136; Last batch loss: 48.1362
HAN:
Is that good or bad?

GAN:
Thanks him.

MAN:
Shhoy mice so tell to sound.

LEIA:
What it.

BOELLIT:
 ENR. That Neppe now alrea wouly spalistemn.

S INK:
What roag!.. sheap have.

REEPEUCMAR:
It and are.

HONG:
You civer conute the won't let ore the corller rige, am though or the moers fraples t

Epoch: 000022
Average_loss: 56.1751; Last batch loss: 47.9021
HAN:
Is that good or bad?

GONDL:
Go you.

JEG. No. What he pentr tell me!

GUNTERAY:
We down't wain't mas.

LEIA:
Culd me Lif imyese sad ow a bumfeed you beath of abousicind.

DUGIETE:
He wild can teop.

LUKE:

Epoch: 000040
Average_loss: 54.3719; Last batch loss: 45.7929
HAN:
Is that good or bad?

GAN:
Mre.

LANDOF TER:
Whow so wery?

LUKE:
ThemE, told I's may sempoosefes it I'll once.  I think burnde.

LEFA:
Reaves, I don't truster I mese out?

LUKE:
Pair Tirnch. You cumming. Gadbon good runger hese. HER... conty you at ham it.

DYO PIL:
At los? Well is forutalk.



Epoch: 000041
Average_loss: 54.2968; Last batch loss: 45.6530
HAN:
Is that good or bad?

GRESPER:
What gety, in back tondar.

SCOANDR:
I a corvelarbot pille noar. I've godiss thould I know my you licm.

SANDY:
Hew it momed reae hiull! The simest a day, acceed yound a sclous orred, whit everaiss to pepsesnticce wanconen thing. No.  focenstod olatetifowitele wil

Epoch: 000042
Average_loss: 54.2235; Last batch loss: 45.5321
HAN:
Is that good or bad?

DGE:
But a not beaste think.

THENE:
The mover if to will isker.  the do sirhtal take ald Dsey?

LANDO:
Oal!

MAVERICK:
I heapbous yeve you say I fatakpopre.  I know a lowes bettan, o